<a href="https://colab.research.google.com/github/abonic92/Analisis-Ventas-Python-Pandas/blob/EDA/clasedata_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis de datos

# ETL (Estracion, transformacion y carga)

### Estracion

In [1]:
from google.colab import drive
import pandas as pd
import chardet

#montamos Google Drive para acceder al archivo
drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/analisis_de_datos-info/ventas.csv'
with open(ruta_archivo, 'rb') as f:
    result = chardet.detect(f.read())

try:
  df = pd.read_csv(ruta_archivo, encoding=result['encoding'], sep=';', decimal=',')
  print('Extración de datos exitosa')
except FileNotFoundError:
  print(f'El archivo no se encuentra en la ruta: {ruta_archivo}')


Mounted at /content/drive
Extración de datos exitosa


### Transformación

In [2]:
# el metodo HEAD muestra las primeras 5 filas
df.head(10)

,Tienda,Marca,Tipo,Gï¿½nero,Talla,Color,Categoria,precio de venta,Fecha,Hora
0,Lima,Asics,WB1820,Femenino,42,Azul,Pantalon,89.00,15/07/2015,07:32:00
1,Lima,Asics,Kayano Single Tab,Usinex,42-44,Azul,Ropa interior,24.99,15/07/2015,07:33:00
2,Lima,Asics,WB1820,Femenino,37,Rosado,Pantalon,89,15/07/2015,07:52:00
3,Lima,Asics,WB2585,Femenino,39,Negro,Pantalon,99,15/07/2015,07:58:00
4,Lima,Asics,WB1820,Femenino,46,Multicolor,Pantalon,89,15/07/2015,08:19:00
5,Lima,Asics,WB1820,Femenino,39,Azul,Pantalon,89,15/07/2015,08:48:00
6,Lima,Asics,MB1878,Masculino,38,Negro,Pantalon,99,15/07/2015,09:07:00
7,Lima,Asics,Hera Deux Single Tab,Femenino,42-44,Negro,Ropa interior,12.99,15/07/2015,09:11:00
8,Lima,Asics,MB1878,Masculino,38,Negro,Pantalon,99,15/07/2015,10:04:00
9,Lima,Asics,MB1878,Masculino,46,Negro,Pantalon,99,15/07/2015,10:06:00


In [3]:
# INFO: este metodo nos da  un resumen de la informacion del DATAFRAME
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Tienda           800 non-null    object
 1   Marca            800 non-null    object
 2   Tipo             800 non-null    object
 3   Gï¿½nero         800 non-null    object
 4   Talla            800 non-null    object
 5   Color            800 non-null    object
 6   Categoria        800 non-null    object
 7   precio de venta  800 non-null    object
 8   Fecha            800 non-null    object
 9   Hora             800 non-null    object
dtypes: object(10)
memory usage: 62.6+ KB


In [4]:
# Renombrar columnas para facilitar el acceso y evitar caracteres especiales
df.rename(columns ={
    'Tienda': 'tienda',
    'Marca': 'marca',
    'Tipo': 'tipo',
    'Gï¿½nero': 'genero',
    'Talla': 'talla',
    'Color': 'color',
    'Categoria': 'categoria',
    'precio de venta': 'precio',
    'Fecha': 'fecha',
    'Hora': 'hora'
}, inplace=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tienda     800 non-null    object
 1   marca      800 non-null    object
 2   tipo       800 non-null    object
 3   genero     800 non-null    object
 4   talla      800 non-null    object
 5   color      800 non-null    object
 6   categoria  800 non-null    object
 7   precio     800 non-null    object
 8   fecha      800 non-null    object
 9   hora       800 non-null    object
dtypes: object(10)
memory usage: 62.6+ KB


In [5]:
# Convertí la columna 'precio' a numérica
df['precio'] = df['precio'].astype(str)
df['precio'] = df['precio'].str.replace(',', '.', regex=True).astype(float)

# Limpiar y unificar los datos de la columna 'genero'
df['genero'] = df['genero'].str.replace('Masculino', 'M', regex=False).str.replace('Femenino', 'F', regex=False).str.replace('Usinex', 'U', regex=False)

# Crear una columna de fecha y hora combinada y convertila al tipo datatime
df['fecha_hora'] = pd.to_datetime(df['fecha'] + ' ' + df['hora'], format='%d/%m/%Y %H:%M:%S')


# Eliminar las columnas originales de fecha y hora
df.drop(columns=['fecha', 'hora'], inplace=True)


print("Transformación de datos exitosa")

df.info()


Transformación de datos exitosa
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   tienda      800 non-null    object        
 1   marca       800 non-null    object        
 2   tipo        800 non-null    object        
 3   genero      800 non-null    object        
 4   talla       800 non-null    object        
 5   color       800 non-null    object        
 6   categoria   800 non-null    object        
 7   precio      800 non-null    float64       
 8   fecha_hora  800 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 56.4+ KB


In [6]:
#tomamois el  primer valor numerico y descartamos el resto
#tomamos el valor de la izquierda si hay un guion
df['talla'] = df['talla'].astype(str).apply(lambda x: x.split('-')[0])

#Ahora convertimos a tipo numero (entero)
df['talla'] = pd.to_numeric(df['talla'], errors='coerce').astype('Int64')


print("Transformación de datos exitosa")

print(set(df['talla']))

print('#######################################################################')
print('#######################################################################')

df.info()



Transformación de datos exitosa
{np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), <NA>}
#######################################################################
#######################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   tienda      800 non-null    object        
 1   marca       800 non-null    object        
 2   tipo        800 non-null    object        
 3   genero      800 non-null    object        
 4   talla       727 non-null    Int64         
 5   color       800 non-null    object        
 6   categoria   800 non-null    object        
 7   precio      800 non-null    float64       
 8   fecha_hora  800 non-null    datetime64[ns]
dtypes: Int64(1), datetime64[ns

In [7]:
print(df['categoria'].unique())
print(df['tienda'].unique())
print(df['marca'].unique())

print('#######################################################################')
print('#######################################################################')

print(df.groupby('categoria')['talla'].unique())

print(df.groupby('tienda')['marca'].unique())

print('#######################################################################')
print('#######################################################################')

print('Prio total  por marcas  en cada tienda')
#print(df.groupby(['tienda','marca'])['precio'].sum().reset_index())

['Pantalon' 'Ropa interior' 'Zapatillas' 'Sujetadores']
['Lima' 'Provincias']
['Asics' 'Nike' 'Adidas']
#######################################################################
#######################################################################
categoria
Pantalon         [42, 37, 39, 46, 38, 40, 41, 45, 43, 36, 44]
Ropa interior                        [42, 38, 36, 40, 46, 44]
Sujetadores                                            [<NA>]
Zapatillas       [44, 39, 42, 37, 38, 41, 40, 43, 45, 46, 36]
Name: talla, dtype: object
tienda
Lima          [Asics, Nike, Adidas]
Provincias    [Asics, Adidas, Nike]
Name: marca, dtype: object
#######################################################################
#######################################################################
Prio total  por marcas  en cada tienda


In [8]:
# Calcular la mediana de la columna 'precio de venta'
mediana_precio = df['precio'].median()

# Calcular el promedio de la mediana (que será el mismo valor de la mediana)
promedio_mediana = mediana_precio

print(f"La mediana del precio de venta es: {mediana_precio}")
print(f"El promedio de la mediana del precio de venta es: {promedio_mediana}")

La mediana del precio de venta es: 89.0
El promedio de la mediana del precio de venta es: 89.0


In [9]:
#DESCRIBE: Generar estadisticas descriptivas de las columnas numericas
df.describe()

,talla,precio,fecha_hora
count,727.0,800.000000,800
mean,41.0,83.175112,2015-07-18 07:26:51.975000064
min,36.0,12.990000,2015-07-15 07:32:00
25%,38.0,21.000000,2015-07-16 13:04:30
50%,41.0,89.000000,2015-07-17 16:28:00
75%,44.0,99.000000,2015-07-19 23:36:00
max,46.0,239.000000,2015-07-22 23:06:00
std,3.238882,55.146881,NaN


### Carga (Load)

In [10]:
# definir la ruta de salida para guardar el nuevo CSV
ruta_salida = '/content/drive/MyDrive/analisis_de_datos-info/ventas_transformadas2.csv'

# Guardar el dataframe transformado en un nuevo archivo CSV en Drive
df.to_csv(ruta_salida, index=True, index_label='id', sep=',')

print(f'Archivo CSV guardado en: {ruta_salida}')

Archivo CSV guardado en: /content/drive/MyDrive/analisis_de_datos-info/ventas_transformadas2.csv
